In [ ]:
import gc, os, sys, traceback
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
def write_error(e):
    with open('error.log', 'a') as f:
        f.write(f'\nERROR: {str(e)}\n')
        traceback.print_exc(file=f)
    print(f'Error: {e}')
sys.excepthook = lambda t, v, tb: write_error(v)

In [ ]:
!apt-get update -y && apt-get install -y espeak-ng
!pip install phonemizer soundfile librosa resemble-perth s3tokenizer conformer
!pip install git+https://github.com/rsxdalv/chatterbox.git --no-deps

In [ ]:
TEXT = "Success for Chatterbox too!"
VOICE_ID = 'af_bella'
OUTPUT_FILE = "chatterbox_success.wav"

In [ ]:
import torch
from chatterbox import ChatterboxTTS
print('Loading Chatterbox...')
tts = ChatterboxTTS.from_pretrained(device='cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(f'Generating with chatterbox...')
import numpy as np
import soundfile as sf
audio_data = tts.generate(TEXT)
if hasattr(audio_data, 'detach'): audio_data = audio_data.detach()
if hasattr(audio_data, 'cpu'): audio_data = audio_data.cpu().numpy()
if isinstance(audio_data, tuple): audio_data = audio_data[0]
if hasattr(audio_data, 'ndim') and audio_data.ndim > 1:
    if audio_data.shape[0] == 1: audio_data = audio_data.squeeze(0)
    elif audio_data.shape[-1] == 1: audio_data = audio_data.squeeze(-1)
sf.write(OUTPUT_FILE, audio_data, 24000)
print(f'Done, saved to {OUTPUT_FILE}, shape {getattr(audio_data, "shape", "?")}')

In [ ]:
del tts
torch.cuda.empty_cache()
print('Cleaned up GPU')